In [13]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [164]:
ativo = 'wdo_5'
df = pd.read_csv(f'{ativo}.csv')

In [165]:
tempo_fechar = -5
df['MA_9'] = ta.EMA(df['close'], timeperiod = 9)
df['MA_9_diff_value'] = df['MA_9'].diff(periods = 1)
df['MA_9_tend'] = df['MA_9_diff_value'].apply(lambda x: 1 if x > 0 else -1)
df['MA_9_tend_diff'] = df['MA_9_tend'].diff()

df['acao'] = df['MA_9_tend_diff'].apply(lambda x: 'call' if x == 2 else 'sell' if  x ==-2 else 0)
df['preco'] = df.apply(lambda x: x['high'] if x['acao']=='call' else x['low'] if x['acao'] == 'sell' else 0, axis =1)
df['stop'] = df.apply(lambda x: x['low'] if x['acao']=='call' else x['high'] if x['acao'] == 'sell' else 0, axis =1)

df['realizacao_simplista'] = df['close'].shift(tempo_fechar)

df = df.dropna()


df_acao = df[df['acao'] != 0]



def cal_result(x):
    if x['acao'] == 'call':
        if x['realizacao_simplista'] < x['stop']:
            resultado = x['stop'] - x['preco']
            return resultado
        else:
            resultado = x['realizacao_simplista'] - x['preco']
            return resultado
    if x['acao'] == 'sell':
        if x['realizacao_simplista'] > x['stop']:
            resultado = x['stop'] - x['preco']
            return resultado
        else:
            resultado = x['realizacao_simplista'] - x['preco']
            return resultado

df_acao['resultado_valor'] = df_acao.apply(cal_result, axis = 1)
df_acao['resultado_binario'] = df_acao['resultado_valor'].apply(lambda x: 1 if x > 0 else 0)

In [166]:
compra = df_acao[df_acao['acao'] =='call']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='call'].shape[0]
venda = df_acao[df_acao['acao'] =='sell']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='sell'].shape[0]

compra_result = df_acao[df_acao['acao'] =='call']['resultado_valor'].sum()
venda_result = df_acao[df_acao['acao'] =='sell']['resultado_valor'].sum()

print(f'Venda: {venda}% -- Compra: {compra}%')
print(f'Venda: {venda_result} -- Compra: {compra_result}')

Venda: 0.5247933884297521% -- Compra: 0.3920220082530949%
Venda: -5.629999999999981 -- Compra: -9.749999999999986


In [162]:
df_acao.iloc[-100:].head(50)

,data,open,high,low,close,vol,MA_9,MA_9_diff_value,MA_9_tend,MA_9_tend_diff,acao,preco,stop,realizacao_simplista,resultado_valor,resultado_binario
7358,2022-12-20 09:28:00,5322.5,5323.0,5319.5,5320.5,4420,5321.411766,-0.227941,-1,-2.0,sell,5319.5,5323.0,5326.5,3.5,1
7359,2022-12-20 09:29:00,5320.5,5322.5,5319.0,5321.5,3393,5321.429413,0.017647,1,2.0,call,5322.5,5319.0,5327.0,4.5,1
7375,2022-12-20 09:45:00,5330.5,5333.0,5330.0,5330.5,7627,5330.569637,-0.017409,-1,-2.0,sell,5330.0,5333.0,5329.0,-1.0,0
7377,2022-12-20 09:47:00,5328.5,5332.0,5328.0,5330.5,6639,5330.224568,0.068858,1,2.0,call,5332.0,5328.0,5331.0,-1.0,0
7378,2022-12-20 09:48:00,5330.5,5332.0,5329.0,5329.5,4165,5330.079654,-0.144914,-1,-2.0,sell,5329.0,5332.0,5327.5,-1.5,0
7382,2022-12-20 09:52:00,5329.0,5331.5,5329.0,5331.0,6391,5329.711026,0.322243,1,2.0,call,5331.5,5329.0,5329.5,-2.0,0
7383,2022-12-20 09:53:00,5330.5,5331.0,5327.5,5327.5,3684,5329.268821,-0.442205,-1,-2.0,sell,5327.5,5331.0,5328.0,0.5,1
7387,2022-12-20 09:57:00,5328.5,5330.5,5328.0,5329.5,6361,5328.384509,0.278873,1,2.0,call,5330.5,5328.0,5329.5,-1.0,0
7388,2022-12-20 09:58:00,5329.5,5331.5,5328.0,5328.0,6925,5328.307607,-0.076902,-1,-2.0,sell,5328.0,5331.5,5328.5,0.5,1
7389,2022-12-20 09:59:00,5328.5,5332.0,5328.5,5332.0,9784,5329.046086,0.738479,1,2.0,call,5332.0,5328.5,5325.5,-3.5,0
